In [0]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, color


SF = 2   #scaling factor, i.e. how much do you want to scale your image (here, twice of original image)
learning_rate = 0.01
ip_height = 160
ip_width = 240

In [0]:
temp1 = io.imread_collection('home/wazir/Desktop/baboonLR.png')
temp2 = io.imread_collection('home/wazir/Desktop/baboonHR.png')

xx = np.array([images for i,images in enumerate(temp1)],dtype =np.float32).reshape(-1,160,240,1)
yy = np.array([images for i,images in enumerate(temp2)],dtype= np.float32).reshape(-1,1,320*480)


X = tf.placeholder(tf.float32, shape=(None, 160, 240, 1),name="X" )        # input
Y = tf.placeholder(tf.float32,shape=(None, 1, 320*480),name="Y" )      # actual output

#print(xx.shape)

In [0]:
def conv2d(x, W, b, strides=1, act_fn = 1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    if act_fn ==1:
        return tf.nn.relu(x)
    else:
        return x

def dil_conv2d(x, W, b, rate=2):
    # Dilation Conv2D wrapper, with bias and but no activation
    x = tf.nn.atrous_conv2d(x, W , rate, padding='SAME')
    x = tf.nn.bias_add(x, b)
    return x

In [0]:
# Store layer's weight & bias   ------   weights here are equivalent to 'filters' in tensorflow

def CNN(X):
    weights = {
        # 5x5 conv, 1 input, 16 outputs                             FEATURE REP.
        'wc1': tf.Variable(tf.random_normal([5, 5, 1, 16]),name='wc1',dtype=tf.float32),
    
        # 5x5 conv, 56 inputs, 8 outputs                           SHRINKING
        'wc2': tf.Variable(tf.random_normal([5, 5, 16, 8]),'wc2',dtype=tf.float32),
    
        # 5x5 conv, 8 inputs, 8 outputs                           NON-LINEAR MAPPING
        'wc3': tf.Variable(tf.random_normal([5, 5, 8, 8]), 'wc3',dtype=tf.float32),
        # 3X3 dilation conv, 12 inputs, 12 outputs                  NON-LINEAR MAPPING
        'wd1': tf.Variable(tf.random_normal([3, 3, 8,8]),'wd1',dtype=tf.float32),
        # 5x5 conv, 12 inputs, 12 outputs                           NON-LINEAR MAPPING
        'wc4': tf.Variable(tf.random_normal([5, 5, 8,8]),'wc4',dtype=tf.float32),
        # 3X3 dilation conv, 12 inputs, 12 outputs                  NON-LINEAR MAPPING
        'wd2': tf.Variable(tf.random_normal([3, 3, 8,8]),'wd2',dtype=tf.float32),
        # 5x5 conv, 12 inputs, 12 outputs                           NON-LINEAR MAPPING
        'wc5': tf.Variable(tf.random_normal([5, 5, 8,8]),'wc5',dtype=tf.float32),

        # 5x5 conv, 12 inputs, 56 outputs                           EXPANSION
        'wc6': tf.Variable(tf.random_normal([5, 5, 8, 16]),'wc6',dtype=tf.float32),

        #the last convolution, 16 inputs, 2 outputs
        'wc7': tf.Variable(tf.random_normal([5, 5, 16, 1]),'wc7',dtype=tf.float32),
        'wc8': tf.Variable(tf.random_normal([4, 4, 1, 16]),'wc8',dtype=tf.float32)
    }

    biases = {
        'bc1': tf.Variable(tf.random_normal([16]),dtype=tf.float32),    #FEATURE REPRESENTATION

        'bc2': tf.Variable(tf.random_normal([8]),dtype=tf.float32),     #SHRINKING

        'bc3': tf.Variable(tf.random_normal([8]),dtype=tf.float32),     #NON-LINEAR MAPPING
        'bd1': tf.Variable(tf.random_normal([8]),dtype=tf.float32),     #NON-LINEAR MAPPING
        'bc4': tf.Variable(tf.random_normal([8]),dtype=tf.float32),     #NON-LINEAR MAPPING
        'bd2': tf.Variable(tf.random_normal([8]),dtype=tf.float32),     #NON-LINEAR MAPPING
        'bc5': tf.Variable(tf.random_normal([8]),dtype=tf.float32),     #NON-LINEAR MAPPING

        'bc6': tf.Variable(tf.random_normal([16]),dtype=tf.float32),    #EXPANSION

        'bc7': tf.Variable(tf.random_normal([1]),dtype=tf.float32)     #DECONV LAYER (not reqd.)
    }


    X = tf.cast(tf.reshape(X, shape=[-1, 160, 240, 1]),tf.float32)

    c1 = conv2d(X, weights['wc1'], biases['bc1'])    #FEATURE REP

    c2 = conv2d(c1, weights['wc2'], biases['bc2'])          #SHRINKING

    c3 = conv2d(c2, weights['wc3'], biases['bc3'])          #NON-LINEAR MAPPING
    d1 = dil_conv2d(c3, weights['wd1'], biases['bd1'])
    c4 = conv2d(d1, weights['wc4'], biases['bc4'])
    d2 = dil_conv2d(c4, weights['wd2'], biases['bd2'])
    c5 = conv2d(d2, weights['wc5'], biases['bc5'])

    c6 = conv2d(c5, weights['wc6'], biases['bc6'], act_fn =0)     #EXPANSION (LOCAL QUEUE JUMPING)
    c6 = tf.add(c1,c6)
    c6 = tf.nn.relu(c6)
    
    op_image =tf.contrib.layers.conv2d_transpose(c6, 1, 4, 2, padding="same")
    op_image = tf.reshape(op_image, ( -1,1, 320*480),"op_image")
    
    return op_image

In [0]:
def train_cnn(X):
    prediction = CNN(X)
    #mse
    loss = tf.reduce_mean(tf.pow(tf.subtract(prediction, Y), 2.0))
    #optimizer
    train = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(5):
            epoch_loss = 0
            
            c = sess.run(loss, feed_dict={X: xx, Y: yy})
            b = sess.run(train, feed_dict={X: xx, Y: yy})
            print('Epoch', epoch, 'completed out of 10  loss:', c)


In [6]:
train_cnn(X)

Epoch 0 completed out of 10  loss: nan
Epoch 1 completed out of 10  loss: nan
Epoch 2 completed out of 10  loss: nan
Epoch 3 completed out of 10  loss: nan
Epoch 4 completed out of 10  loss: nan
